## DESE61003 - Audio Experience Design
# Week 5 - Data-Driven Sound Recognition

In [ ]:
from disiple.signals import AudioSignal, Spectrum, PowerSpectrum, Spectrogram, TimeSignal
from disiple.util import download_and_extract_archive, download_file
import numpy as np
from scipy import signal
from pathlib import Path

In this session, we'll go over some basic machine learning concepts to show you how sounds can be recognised based on previously seen examples. In the previous session, we saw that various sound recognition scenarios boil down to a classification problem. Such classification problems appear in many application domains, and the algorithms that can learn how to classify sounds are the same as used in other domains. We'll be using the [`scikit-learn`](https://scikit-learn.org) library, which comes with many excellent [getting started guides](https://scikit-learn.org/stable/getting_started.html), [tutorials](https://scikit-learn.org/stable/tutorial/basic/tutorial.html) and [examples](https://scikit-learn.org/stable/auto_examples/index.html), but they are not focussed on audio. Therefore the aim of this notebook is to give a practical example applied to audio, such that you can afterwards make better sense of the scikit-learn documentation. Of course you won't become a machine learning expert after one session, but hopefully you'll get enough understanding to start playing around with some code.

One of the biggest hurdles when getting started with `scikit-learn` or any other machine learning library is getting your own data into a format that can be used as input for the algorithms. Most of this notebook will therefore be spent on data preparation. Afterwards a few lines of `scikit-learn` code will do the heavy lifting. These data preparation techniques are things you just need to be shown, so unlike previous notebooks, there won't be any fill-in questions and activities throughout the notebook, just a line-by-line explanation of what's going on. Instead a couple of larger tasks will be suggested at the end, which will involve modifying the entire notebook.

### 1. Getting example audio files

Machine learning requires data to learn from, usually the more the better, so we will start by downloading some audio files. First we'll choose a common folder to store everything in. On MyBinder, the notebooks are stored in a subfolder and we want to place the audio files in a folder on the same level as the notebooks folder. Therefore we go up one level in the hierarchy with the `..`, then use or create a folder in the parent directory of this notebook's folder. When you're running this notebook locally, however, it might be stored in the root folder of Jupyter. In that case there is no parent folder to go to, and this will cause errors later on. So if you're running this notebook on your own computer, and you're not entirely sure what your root Jupyter folder is or what that means, it will be safer to change the string below to `./data`. Then you'll store all audio in a subfolder called `data`.

In [ ]:
audio_dir = Path('../data')

Notice that we've made `audio_dir` a variable of type [`pathlib.Path`](https://docs.python.org/3/library/pathlib.html). This is a class of Python's built-in library which provides filesystem functionality that we can use later. Much like we indicated that a NumPy array is not just any array, but contains audio data by passing it to the constructor of `AudioSignal`, creating a `Path` from a string shows that this is not any string, but a string representing a path in the filesystem. In return you get filesystem specific additional functionality, just like you got playback functionality for NumPy arrays passed to `AudioSignal`.

Like the previous session, we're getting instrument samples from the [University of Iowa collection](https://theremin.music.uiowa.edu/MIS.html). Extending our trumpet vs flute example, we're now getting zip files containing recordings of all notes that can be played on each instrument.

In [ ]:
trumpet_url = 'https://theremin.music.uiowa.edu/sound%20files/MIS%20Pitches%20-%202014/Brass/BbTrumpet/Trumpet.novib.ff.stereo.zip'
download_and_extract_archive(trumpet_url, audio_dir / 'trumpet', verbose=True)

In [ ]:
flute_url = 'https://theremin.music.uiowa.edu/sound%20files/MIS%20Pitches%20-%202014/Woodwinds/Flute/Flute.nonvib.ff.stereo.zip'
download_and_extract_archive(flute_url, audio_dir / 'flute', verbose=True)

Note that we used here some `Path`-specific functionality, the [`/` slash operator](https://docs.python.org/3/library/pathlib.html#operators) which is not defined for regular strings, but allows to concatenate paths while taking care of operating system specifics (so it's a bit more intelligent than the `+` string operator).

We've now downloaded and extracted instrument samples to both the `trumpet` and `flute` folder of our audio directory and can show their contents.

In [ ]:
%ls {audio_dir}/trumpet

In [ ]:
%ls {audio_dir}/flute

In order to train machine learning algorithms, or more specifically the _supervised learning_ approach we're following to create a classifier, we don't just need audio files but we also need to describe what they contain. This description of the contents (known as _class label_) is what they algorithm will (ideally) learn. To make things easy, we're going to rely on the fact that we have grouped multiple examples of the same sound in one folder and that the name of that folder is a description of its content. This is already the case for our `flute` and `trumpet` folders. We could also reorganise our files to put the same note played by different instruments in the same folder and name that folder after that common note. After training (and in the optimal scenario), our classifier would then be able to recognise notes, not instruments. There are no limitations on how to describe you audio, it does not even have to be meaningful, but the audio files in one category should have some property in common though, otherwise it is unlikely that the machine learning algorithm can learn something meaningful. It is therefore perfectly fine to name a folder of trumpet samples "orange" and a folder of flute samples "green", if your intention is to have other trumpet samples be recognised as "orange". If you randomly assign audio to folders, however, the label it will return for other audio will be equally random.

To keep our code generic for later adaptations to other classes, we will from now on talk about `class1` and `class2`. In this case, their labels correspond to "trumpet" and "flute". The audio files for both classes have ".aif" extensions, so we store that in a variable in order to make switching to datasets in a different format easier. `AudioSignal`, which we will use to read those files, supports files in `.wav`, `.mp3`, `.flac`, `.ogg` and `.aif` format (no `.m4a` or `mp4` unfortunately).

In [ ]:
class1_label = 'trumpet'
class2_label = 'flute'
file_extension = '.aif'

We rely on the fact that the label for each class is also the name of the folder to construct the folder name.

In [ ]:
class1_dir = audio_dir / class1_label
class2_dir = audio_dir / class2_label

Then search for all files with the given audio extension in those folders, and get a list of file paths back.

In [ ]:
class1_files = list(class1_dir.glob('*'+file_extension))
class2_files = list(class2_dir.glob('*'+file_extension))

We then create `AudioSignal`s for each of those files on disk and store them in a list.

In [ ]:
class1_audio = [AudioSignal(str(x), samplerate=8000) for x in class1_files]
class2_audio = [AudioSignal(str(x), samplerate=8000) for x in class2_files]

We can check how many files we read by printing the length of each list.

In [ ]:
len(class1_audio), len(class2_audio)

Having the same number of examples for all classes makes it generally easier for a classifier to learn. We've got a couple more examples of a flute, but it's close enough.

Each element of the list is now an `AudioSignal`, so we can play back a random example to check.

In [ ]:
idx = 5
print(class1_label)
class1_audio[idx].play()
print(class2_label)
class2_audio[idx].play()

To make further processing easier, we're going to join both lists of classes into a single list.

In [ ]:
all_audio = class1_audio + class2_audio
len(all_audio)

Now that we put all audio together in one list, we still want to keep track which file corresponds to which label. Therefore we create another list of the same length with the corresponding labels. We know that the first files in the list are of class 1 and the last of class 2, so we simple repeat the class label as many times as there are files and join everything together.

In [ ]:
all_labels = [class1_label] * len(class1_audio) + [class2_label] * len(class2_audio)

### 2. Calculating features

In theory, we could simply use the time-domain representation of these audio files as input to train a classifier (after we made sure all files have the same length). The autonomous training of the classification algorithm does not guarantee it will be successful though, so any human help to make its task easier is welcome. The classifier might be able to come up with a more complex decision process than just thresholding a value, but in practice a good feature representation is still needed. This feature representation can be higher-dimensional than for a pure knowledge-based classification, as the most powerful algorithms will discover ways to combine multiple features and ignore others to come to the best decision. In essence, the learning process comes up with its own thresholding approach, not just based on a single value but considering many dimensions at the same time.

To distinguish between instruments, our hypothesis will be that a spectral representation will be most appropriate. Here we will use the raw signal representation directly as input, but it is certainly worth considering if further features could be derived from it instead. All audio files still have different lengths though, so we first calculate the length in samples of the longest file.

In [ ]:
all_lengths = [len(x) for x in all_audio]
max_length = max(all_lengths)

We then caclulate a spectrum for each audio file, but request all to have as many bins as the longest file. Not only will this give features of equal length for all files, it also ensures that the frequency resolution is equal for all files, thereby making the features comparable between files.

In [ ]:
all_spectra = [Spectrum(x, num_bins=max_length) for x in all_audio]

We now have a list that contains as many spectra as there were audio files, but we also know that all spectra have the same size. Therefore we can transform this list of lists into a two-dimensional NumPy array (think of it as a matrix, although there are small conceptual differences in terms of arithmetics between a matrix and 2D array).

In [ ]:
all_spectra_array = np.array([x.magnitude for x in all_spectra])
all_spectra_array.shape

We did have use the `.magnitude` method to get the magnitudes of each spectrum out of their `Spectrum` object as an array, but the end result is a 2D array with each example occupying a row. Each column represents a frequency bin in the spectra, and is considered an input feature for the classifier. Such 2D array of `num_examples x num_features` size is exactly what `scikit-learn` expects as input.

### 3. Training a nearest neighbour classifier

Now that we have our audio files transformed to the right input format, we can start using `scikit-learn`. We will start this demonstration with a [nearest neighbour classifier](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm). This machine learning approach does not really _learn_ anything about the given examples, it does not attempt to discover commonalities between examples of a class and differences between examples of different classes. Rather it remembers all examples and checks which ones are the closest to a newly presented file. The label that is most prevalent among the closest examples is then assigned to the new file. The only parameter is therefore how many close neighbours are being considered.

All machine learning models in `scikit-learn` have a consistent way to be used, which makes it easy to swap between algorithms. A model is created by calling its particular constructor (here `KNeighborsClassifier` which is loaded from `sklearn.neighbors`), then trained by calling its `.fit()` method with as first argument the features as a `num_examples x num_features` array and as second argument its labels as a list or array of length `num_examples`.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=3)
model.fit(all_spectra_array, all_labels)

Here we've set the number of closest neighbours to consider to 3. Remember that you can always check the signature of the constructor for help about the various options by appending `?` to the class name.

In [ ]:
KNeighborsClassifier?

Now that the model is trained, we can ask it to predict the label for any file. You use the `.predict()` method for that, which you again pass a 2D array of `num_examples x num_features`, but no labels since you want the model to predict these. For instance, we can ask the model what it thinks the last example was.

In [ ]:
model.predict(all_spectra_array[-1:])

That sample was a flute indeed, but we don't need to ask the model for predictions one-by-one, also in `.predict()` can `num_samples` be more than one, so let's try our whole collection.

In [ ]:
predictions = model.predict(all_spectra_array)
predictions

We can see that most flute samples are correctly recognised as flute, but also quite some trumpet samples. We don't need to tally the score manually though, there's a function for that. From `sklearn.metrics` we load `accuracy_score`, which gives us the percentage of correct predictions. If you pass the known correct labels as first argument and the predictions as second,  the score will be returned as a number between zero and one. Many other [metrics](https://scikit-learn.org/stable/modules/model_evaluation.html) are available, but accuracy is a good start for evaluating classification.

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(all_labels, predictions)

A score of 82.7% is not too bad if you consider that a random guess would be around 50%, but it also means our classifier is far from perfect. A single number does not give much information about where the classification went right or wrong though, so we'll use a more detailled metric.

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
ConfusionMatrixDisplay.from_predictions(all_labels, predictions)

A [confusion matrix](https://en.wikipedia.org/wiki/Confusion_matrix) shows how many times each class is category is classified correctly on its diagonal, but also the type of mistakes that are made. We see that trumpets are many times more recognised as flute than the other way around. Overall the classifier thinks the entire collection consists of 64% (48/75), whereas in reality it is only 52% (39/75).

### 4. Using a train and a test set

The evaluation we just performed is unlikely to give an accurate view of how well the classifier would perform when presented with new audio files. After all, we used the same files to evaluate with as those we used for training. A file therefore always had its own spectrum as closest neighbour. The only reason why we didn't get a score of 100% is because there were two more neighbouring spectra that influenced the vote on the type of instrument. If the second and third closest spectrum both were of the other class, only then would the correct answer be overruled. So if the number of closest neighbours was set to 1, the recognition would be perfect.

In reality, a newly presented sound would not be present in the collection used for training, so it's likely to fare worse than the predicted performance of 82% correct. A way to simulate this situation is by not using all the audio in our collection for training, but to keep a fraction apart for evaluation purposes only. That means there will be less data available for training, so the reported performance could be an underestimate of the performance comparing to using the whole collection, but it is better to make a conservative estimate. The [`train_test_split`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function takes features and labels and based on the value of the `test_size` parameter splits them randomly into training features and labels and test features and labels.

In [ ]:
from sklearn.model_selection import train_test_split
train_spectra, test_spectra, train_labels, test_labels = train_test_split(all_spectra_array, all_labels, test_size=0.25)

The size of test vs train test is a delicate balance. On the one hand, you don't want to make the train set too small because it will lead to underestimating the performance compared to using the whole collection, whereas on the other hand you don't want the test set to be too small either because you don't want to draw any conclusions from just a few examples, which can have considerable variation. Therefore you want your test set to be large enough to average out unrepresentative scores. A common value for the size of the test set is 25% of the overall data, but trying a range of values and seeing how the scores respond can be most insightful.

With our train and test sets ready, we can now perform a more representative experiment.

In [ ]:
model = KNeighborsClassifier(n_neighbors=3)
model.fit(train_spectra, train_labels)
test_predictions = model.predict(test_spectra)
ConfusionMatrixDisplay.from_predictions(test_labels, test_predictions)
accuracy_score(test_labels, test_predictions)

The split into train and test set is done randomly, so the result of the experiment will vary but will likely be below 82%. To mitigate the source of variation coming from random splitting, the the whole process of splitting, training and evaluating can be repeated multiple times and evaluation averaged until convergence is reached.

### 5. Trying other types of classifier

Since the programming interface of `scikit-learn` is so consistent, it is easy to swap around different types of classifiers. To that end, a function is defined below and executed with a [Naive Bayes](https://scikit-learn.org/stable/modules/naive_bayes.html) classifier and a [Support Vector Machine](https://scikit-learn.org/stable/modules/svm.html). The scores are non-deterministic again because of the random split, but overall they represent quite an improvement over the nearest neighbour classifier.

In [ ]:
def evaluate_model(model, features, labels, test_size=0.25):
    train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=test_size)
    model.fit(train_features, train_labels)
    test_predictions = model.predict(test_features)
    ConfusionMatrixDisplay.from_predictions(test_labels, test_predictions)
    return accuracy_score(test_labels, test_predictions)

In [ ]:
from sklearn.naive_bayes import GaussianNB
evaluate_model(GaussianNB(), all_spectra_array, all_labels)

In [ ]:
from sklearn.svm import SVC
evaluate_model(SVC(), all_spectra_array, all_labels)

### 6. Tasks to try

Now that you made it all the way through, you should be ready to try some of the following tasks

- Write a function that executes the splitting, training, evaluating process repeatedly and averages the score. How many iterations do you need to run before the average stays somewhat stable?
- Try classifiying other sounds and/or labels. Some of the instruments in the University of Iowa collection are recorded with multiple playing techniques (e.g. [bowed or pizzicato string instruments](https://theremin.music.uiowa.edu/MIS-Pitches-2012/MISViolin2012.html), [woodwinds with or without vibrato](https://theremin.music.uiowa.edu/MIS-Pitches-2012/MISFlute2012.html)). Have you considered recognising playing technique of an instrument? Do you think it would be easier or harder than distinguishing between instruments?
- Experiment with other representations and/or features as input to a classifier. Have you tried using the spectral centroid, zero-crossing rate or anything else you worked on last session as feature? If you want to make multiple time-domain signals have the same length, have a look at [`np.pad()`](https://numpy.org/doc/stable/reference/generated/numpy.pad.html). If you want to transform a 2D array such as a spectrogram into a single row, have a look at NumPy's [`.reshape()`](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.reshape.html) method.
- You can pass multiple features as input to a classifier, which will then decide the ones that are most useful (although it can also be distracted by the extra data, so it's not guaranteed to lead to an improvement). Have you tried combining different features, such as a time-domain one and a frequency-domain one?
- Add a third class or even more.
- Test more [classifiers from `scikit-learn`](https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html) and/or play around with their parameters.
- Even though data-driven, the types of classifiers available in `scikit-learn` still rely heavily on the choice and development of appropriate features. This is not optimal because it requires human expertise and foresight to handle a wide variety of data. Deep learning approaches, which have seen an exponential growth over the last ten years, aim to remedy this by automatically discovering the best features themselves (but require a whole other set of human expertise). Their inputs are usually direct signal representations, no further derived (and reductive) features because that would limit their freedom to discover new ones. If you're confident with Python, have previous machine learning experience and are motivated, you can follow a [speech command recognition tutorial notebook on the Tensorflow website](https://www.tensorflow.org/tutorials/audio/simple_audio).  